# Extração dos dados dos RREO dos pdfs do SIOP

## Teste inicial com arquivos individuais de 2024

In [1]:
import fitz
import pandas as pd
import os
import re

In [4]:
import re
import pandas as pd
from pathlib import Path
import unicodedata

# Dicionário de código IBGE para nome dos municípios
CODIGO_IBGE_NOMES = {
    "1200013": "Acrelândia",
    "1200054": "Assis Brasil",
    "1200104": "Brasiléia",
    "1200138": "Bujari",
    "1200179": "Capixaba",
    "1200203": "Cruzeiro do Sul",
    "1200252": "Epitaciolândia",
    "1200302": "Feijó",
    "1200328": "Jordão",
    "1200336": "Mâncio Lima",
    "1200344": "Manoel Urbano",
    "1200351": "Marechal Thaumaturgo",
    "1200385": "Plácido de Castro",
    "1200393": "Porto Acre",
    "1200807": "Porto Walter",
    "1200333": "Rio Branco",
    "1200427": "Rodrigues Alves",
    "1200435": "Santa Rosa do Purus",
    "1200500": "Sena Madureira",
    "1200450": "Senador Guiomard",
    "1200690": "Tarauacá",
    "1200708": "Xapuri"
}

# Criar dicionário invertido com nomes formatados para comparação
def normalizar_texto(texto):
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII').lower().strip()

NOME_TO_CODIGO = {normalizar_texto(v): k for k, v in CODIGO_IBGE_NOMES.items()}

def carregar_texto(caminho_txt):
    """Lê o conteúdo de um arquivo TXT extraído de PDF."""
    with open(caminho_txt, "r", encoding="utf-8") as f:
        return f.read()

def extrair_dados_rreo(texto):
    """
    Extrai dados principais do RREO a partir de texto bruto extraído com PyMuPDF.

    Retorna:
        dicionário com dados extraídos
    """
    dados = {
        "Cod_IBGE": None,
        "Nome": None,
        "Ano": None,
        "Receita_Impostos": None,
        "20%_Fundeb": None,
        "Mínimo_MDE": None,
        "Receita_Total_Fundeb": None,
        "Receita_VAAF": None,
        "Receita_VAAT": None,
        "Receita_VAAR": None,
        "Despesa_Total_Fundeb": None,
        "Despesas_Fundeb_Prof_Educacao_Total": None,
        "Despesas_Fundeb_Prof_Infantil": None,
        "Despesas_Fundeb_Prof_Fundamental": None,
        "%_Despesas_Fundeb_Prof_Educacao": None,
        "%_Despesas_VAAT_Ed_Infantil": None,
        "%_Despesas_VAAT_Capital": None
    }

    # Extrair nome do município (aparece após "MUNICÍPIOS\n")
    match_nome = re.search(r"MUNIC[ÍI]PIOS\s*\n([A-ZÇÀ-Ú ]+?)\s*-\s*AC", texto, re.IGNORECASE)
    if match_nome:
        nome_extraido = match_nome.group(1).strip().title()
        nome_normalizado = normalizar_texto(nome_extraido)
        dados["Nome"] = CODIGO_IBGE_NOMES.get(NOME_TO_CODIGO.get(nome_normalizado))
        dados["Cod_IBGE"] = NOME_TO_CODIGO.get(nome_normalizado)

    # Extrair ano
    match_ano = re.search(r"Per[ií]odo de Refer[eê]ncia:\s*\dº Bimestre\/(\d{4})", texto)
    if match_ano:
        dados["Ano"] = int(match_ano.group(1))

    # Extrair Receita de Impostos
    match_impostos = re.search(
        r"3- TOTAL DA RECEITA RESULTANTE DE IMPOSTOS \(1 \+ 2\)\s*(\d{1,3}(?:\.\d{3})*,\d{2})\s*(\d{1,3}(?:\.\d{3})*,\d{2})",
        texto,
        re.IGNORECASE
    )
    if match_impostos:
        dados["Receita_Impostos"] = match_impostos.group(2)

    # Extrair Repasse de 20% do Fundeb (aceita variações e dígitos soltos ao final)
    match_20_fundeb = re.search(
        r"""4-\s*TOTAL\s+DESTINADO\s+AO\s+FUNDEB\s*-\s*
            (equivalente\s+a\s+)?20%\s+DE\s+
            \(\(2\.1\.1\)\s*\+\s*\(2\.2\)\s*\+\s*\(2\.3\)\s*\+\s*\(2\.4\)\s*\+\s*\(2\.5\)
            (?:\s*\+\s*\(2\.7\))?
            \)\)*\s*\d*\s*(?:\r?\n\d+)*\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )
    if match_20_fundeb:
        dados["20%_Fundeb"] = match_20_fundeb.group(3)

    # Extrair valor mínimo a ser aplicado em MDE (com ou sem variações no cabeçalho e nos itens)
    match_mde = re.search(
        r"""5-\s*VALOR\s+M[ÍI]NIMO\s+A\s+SER\s+APLICADO\s+
            (EM\s+MDE\s+)?AL[ÉE]M\s+DO\s+VALOR\s+DESTINADO\s+AO\s+FUNDEB\s*-\s*
            5%\s+DE\s+\(\(2\.1\.1\)\s*\+\s*\(2\.2\)\s*\+\s*\(2\.3\)\s*\+\s*\(2\.4\)\s*\+\s*\(2\.5\)
            (?:\s*\+\s*\(2\.7\))?\)\s*\+\s*
            25%\s+DE\s+\(\(1\.1\)\s*\+\s*\(1\.2\)\s*\+\s*\(1\.3\)\s*\+\s*\(1\.4\)\s*\+\s*\(2\.1\.2\)\s*\+\s*\(2\.6\)
            (?:\s*\+\s*\(2\.7\))?\)\s*\)*\d*\s*(?:\r?\n\d+)*\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )
    if match_mde:
        dados["Mínimo_MDE"] = match_mde.group(3)

    # Extrair Receita Total do Fundeb (Item 9 - primeiro valor monetário após o título)
    match_fundeb_total = re.search(
        r"""9-\s*TOTAL\s+DOS\s+RECURSOS\s+DO\s+FUNDEB\s+DISPON[ÍI]VEIS\s+PARA\s+UTILIZA[ÇC][ÃA]O\s*
            \(6\s*\+\s*8\)\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})  # Primeiro valor
        """,
        texto,
        re.IGNORECASE | re.VERBOSE
    )
    if match_fundeb_total:
        dados["Receita_Total_Fundeb"] = match_fundeb_total.group(1)

    # Extrair valor do repasse do VAAF (6.2 - segundo valor monetário)
    match_vaaf = re.search(
        r"""6\.2-\s*FUNDEB\s*-\s*Complementa[çc][ãa]o\s+da\s+Uni[aã]o\s*-\s*VAAF\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE
    )
    if match_vaaf:
        dados["Receita_VAAF"] = match_vaaf.group(2)

    # Extrair valor do repasse do VAAT (6.3 - segundo valor monetário)
    match_vaat = re.search(
        r"""6\.3-\s*FUNDEB\s*-\s*Complementa[çc][ãa]o\s+da\s+Uni[aã]o\s*-\s*VAAT\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE
    )
    if match_vaat:
        dados["Receita_VAAT"] = match_vaat.group(2)

    # Extrair valor do repasse do VAAR (6.4 - segundo valor monetário)
    match_vaar = re.search(
        r"""6\.4-\s*FUNDEB\s*-\s*Complementa[çc][ãa]o\s+da\s+Uni[aã]o\s*-\s*VAAR\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE
    )
    if match_vaar:
        dados["Receita_VAAR"] = match_vaar.group(2)
    else:
        dados["Receita_VAAR"] = "0,00"

    # Extrair Despesa Total com Recursos do Fundeb (versão 1: item 12 - 4º valor)
    match_despesa_12 = re.search(
        r"""12-\s*TOTAL\s+DAS\s+DESPESAS\s*
            COM\s+RECURSOS\s+DO\s+FUNDEB\s*
            \(10\s*\+\s*11\)\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    # Versão 2: item 10 - com múltiplas quebras de linha e valores fragmentados
    match_despesa_10 = re.search(
        r"""10-\s*TOTAL\s+DAS\s+DESPESAS\s+COM\s+RECURSOS\s+DO\s+FUNDEB\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_despesa_12:
        dados["Despesa_Total_Fundeb"] = match_despesa_12.group(4)
    elif match_despesa_10:
        dados["Despesa_Total_Fundeb"] = match_despesa_10.group(4)

    # Extrair despesas com profissionais da educação básica (versões 10 e 10.1, quarto valor numérico)
    match_prof_educ_10 = re.search(
        r"""10-\s*PROFISSIONAIS\s+DA\s+EDUCA[ÇC][ÃA]O\s+B[ÁA]SICA\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_prof_educ_101 = re.search(
        r"""10\.1-\s*PROFISSIONAIS\s+DA\s+EDUCA[ÇC][ÃA]O\s+B[ÁA]SICA\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_prof_educ_10:
        dados["Despesas_Fundeb_Prof_Educacao_Total"] = match_prof_educ_10.group(4)
    elif match_prof_educ_101:
        dados["Despesas_Fundeb_Prof_Educacao_Total"] = match_prof_educ_101.group(4)

    # Extrair despesas com profissionais da Educação Infantil (4º valor após títulos 10.1 ou 10.1.1)
    match_inf_101 = re.search(
        r"""10\.1-\s*Educa[çc][ãa]o\s+Infantil\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_inf_1011 = re.search(
        r"""10\.1\.1\s*-\s*Educa[çc][ãa]o\s+Infantil\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_inf_101:
        dados["Despesas_Fundeb_Prof_Infantil"] = match_inf_101.group(4)
    elif match_inf_1011:
        dados["Despesas_Fundeb_Prof_Infantil"] = match_inf_1011.group(4)

    # Extrair despesas com profissionais do Ensino Fundamental (4º valor após os títulos 10.2 ou 10.1.2)
    match_fund_102 = re.search(
        r"""10\.2-\s*Ensino\s+Fundamental\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_fund_1012 = re.search(
        r"""10\.1\.2-\s*Ensino\s+Fundamental\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_fund_102:
        dados["Despesas_Fundeb_Prof_Fundamental"] = match_fund_102.group(4)
    elif match_fund_1012:
        dados["Despesas_Fundeb_Prof_Fundamental"] = match_fund_1012.group(4)

    # Extrair percentual das despesas do Fundeb com profissionais da educação (4º valor após item 19 ou 15)
    match_pct_19 = re.search(
        r"""19-\s*M[ÍI]NIMO\s+DE\s+70%\s+DO\s+FUNDEB\s+NA\s+REMUNERA[ÇC][ÃA]O\s+
            DOS\s+PROFISSIONAIS\s+DA\s+EDUCA[ÇC][ÃA]O\s+B[ÁA]SICA\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_pct_15 = re.search(
        r"""15-\s*M[ÍI]NIMO\s+DE\s+70%\s+DO\s+FUNDEB\s+NA\s+REMUNERA[ÇC][ÃA]O\s+
            DOS\s+PROFISSIONAIS\s+DA\s+EDUCA[ÇC][ÃA]O\s+B[ÁA]SICA\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_pct_19:
        dados["%_Despesas_Fundeb_Prof_Educacao"] = match_pct_19.group(4)
    elif match_pct_15:
        dados["%_Despesas_Fundeb_Prof_Educacao"] = match_pct_15.group(4)

    # Extrair percentual de despesas do VAAT na Educação Infantil (indicador IEI - variações nos itens 21, 16)
    match_vaat_21 = re.search(
        r"""21-\s*M[ÍI]NIMO\s+DE\s+15%\s+DA\s+COMPLEMENTA[ÇC][ÃA]O\s+DA\s+UNI[ÃA]O\s+AO\s+FUNDEB\s*-\s*VAAT\s+EM\s+DESPESAS\s+DE\s+CAPITAL\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_vaat_16_a = re.search(
        r"""16\s*-\s*PROPOR[ÇC][ÃA]O\s+DE\s+50%\s+DA\s+COMPLEMENTA[ÇC][ÃA]O\s+DA\s+UNI[ÃA]O\s+AO\s+FUNDEB\s*-\s*VAAT\s+NA\s+EDUCA[ÇC][ÃA]O\s+INFANTIL\s*
            \(INDICADOR\s+IEI\)\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_vaat_16_b = re.search(
        r"""16\s*-\s*PERCENTUAL\s+DA\s+COMPLEMENTA[ÇC][ÃA]O\s+DA\s+UNI[ÃA]O\s+AO\s+FUNDEB\s*-\s*VAAT\s+NA\s+EDUCA[ÇC][ÃA]O\s+INFANTIL\s*
            \(INDICADOR\s+IEI\)\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_vaat_21:
        dados["%_Despesas_VAAT_Ed_Infantil"] = match_vaat_21.group(4)
    elif match_vaat_16_a:
        dados["%_Despesas_VAAT_Ed_Infantil"] = match_vaat_16_a.group(4)
    elif match_vaat_16_b:
        dados["%_Despesas_VAAT_Ed_Infantil"] = match_vaat_16_b.group(4)

    # Extrair % de despesas do VAAT em despesas de capital (itens 21 ou 17 - 4º valor numérico)
    match_vaat_cap_21 = re.search(
        r"""21-\s*M[ÍI]NIMO\s+DE\s+15%\s+DA\s+COMPLEMENTA[ÇC][ÃA]O\s+DA\s+UNI[ÃA]O\s+AO\s+FUNDEB\s*-\s*VAAT\s+EM\s+DESPESAS\s+DE\s+CAPITAL\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_vaat_cap_17 = re.search(
        r"""17-\s*M[ÍI]NIMO\s+DE\s+15%\s+DA\s+COMPLEMENTA[ÇC][ÃA]O\s+DA\s+UNI[ÃA]O\s+AO\s+FUNDEB\s*-\s*VAAT\s+EM\s+DESPESAS\s+DE\s+CAPITAL\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_vaat_cap_21:
        dados["%_Despesas_VAAT_Capital"] = match_vaat_cap_21.group(4)
    elif match_vaat_cap_17:
        dados["%_Despesas_VAAT_Capital"] = match_vaat_cap_17.group(4)


  
  
    # Não excluir essa linha da função!
    return dados


def processar_arquivos_pasta(pasta_txt):
    """
    Itera sobre os arquivos .txt da pasta e extrai os dados de todos os relatórios.
    Retorna um DataFrame contendo os dados válidos extraídos.
    """
    lista_dados = []
    pasta_txt = Path(pasta_txt)

    for caminho in pasta_txt.glob("*.txt"):
        texto = carregar_texto(caminho)
        try:
            dados = extrair_dados_rreo(texto)
            if dados and dados.get("Cod_IBGE") and dados.get("Nome"):
                lista_dados.append(dados)
            else:
                print(f"Aviso: Nome ou código IBGE não encontrado ou erro de parsing em {caminho.name}")
        except Exception as e:
            print(f"Erro ao processar o arquivo {caminho.name}: {e}")
            continue

    df = pd.DataFrame(lista_dados)

    # Ordena apenas se não estiver vazio
    if not df.empty:
        df = df.sort_values(by=["Nome", "Ano"], ascending=[True, True])

    return df


# =========================
# EXEMPLO DE EXECUÇÃO
# =========================
if __name__ == "__main__":
    pasta_arquivos_txt = Path("/home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/")
    pasta_dataframe = Path("/home/mpac/Projetos/Curica/datasets_curica/siope/rreo_df/")

    df_rreo = processar_arquivos_pasta(pasta_arquivos_txt)

    # Ordena as colunas do dataframe
    df_rreo = df_rreo.sort_values(by=["Nome", "Ano"], ascending=[True, True])

    # Exporta para CSV
    caminho_saida = pasta_dataframe / "df_rreo_teste.csv"
    df_rreo.to_csv(caminho_saida, index=False, encoding="utf-8-sig", sep=';')
    print(f"Dados extraídos com sucesso. Arquivo salvo em '{caminho_saida}'.")


Dados extraídos com sucesso. Arquivo salvo em '/home/mpac/Projetos/Curica/datasets_curica/siope/rreo_df/df_rreo_teste.csv'.


## Teste de extração recursiva

In [1]:
import fitz  # PyMuPDF
import os
from pathlib import Path

def extrair_texto_pdf(caminho_pdf, caminho_saida_txt):
    """
    Extrai o texto de um PDF e salva em um arquivo .txt.

    Parâmetros:
    ------------
    caminho_pdf : str ou Path
        Caminho completo do arquivo PDF de entrada.
    caminho_saida_txt : str ou Path
        Caminho onde o arquivo de texto será salvo.
    """
    caminho_pdf = Path(caminho_pdf)
    caminho_saida_txt = Path(caminho_saida_txt)

    # Verifica se o arquivo PDF existe
    if not caminho_pdf.exists():
        raise FileNotFoundError(f"Arquivo PDF não encontrado: {caminho_pdf}")

    # Abre o PDF usando PyMuPDF
    doc = fitz.open(caminho_pdf)
    texto_extraido = ""

    # Itera sobre cada página do PDF e extrai o texto
    for pagina_num, pagina in enumerate(doc, start=1):
        texto_extraido += f"\n\n==== PÁGINA {pagina_num} ====\n"  # Marca a página no texto
        texto_extraido += pagina.get_text()  # Extrai o conteúdo textual da página

    # Salva o texto extraído em um arquivo .txt com codificação UTF-8
    caminho_saida_txt.parent.mkdir(parents=True, exist_ok=True)  # Garante que o diretório de saída exista
    with open(caminho_saida_txt, "w", encoding="utf-8") as f:
        f.write(texto_extraido)

    print(f"Texto extraído com sucesso para: {caminho_saida_txt}")

def processar_pdfs_recursivamente(pasta_pdf, pasta_saida_txt):
    """
    Itera sobre os arquivos PDF no diretório especificado e salva os textos extraídos em arquivos .txt.

    Parâmetros:
    ------------
    pasta_pdf : str ou Path
        Caminho para a pasta onde estão os arquivos PDF originais.
    pasta_saida_txt : str ou Path
        Caminho para a pasta onde os arquivos .txt extraídos serão salvos.
    """
    pasta_pdf = Path(pasta_pdf)
    pasta_saida_txt = Path(pasta_saida_txt)

    pasta_saida_txt.mkdir(parents=True, exist_ok=True)  # Cria a pasta de saída se ela não existir

    # Itera sobre os arquivos da pasta PDF
    for caminho_pdf in pasta_pdf.glob("RREO_*.pdf"):
        nome_base = caminho_pdf.stem  # Nome do arquivo sem extensão
        nome_saida = nome_base + "_extraido.txt"
        caminho_saida = pasta_saida_txt / nome_saida

        extrair_texto_pdf(caminho_pdf, caminho_saida)

# =========================
# EXECUÇÃO DIRETA DO SCRIPT
# =========================
if __name__ == "__main__":
    # Define os caminhos de entrada e saída usando Path (adaptável a outros ambientes)
    raiz_projeto = Path("/home/mpac/Projetos/Curica/datasets_curica/siope")
    pasta_entrada_pdf = raiz_projeto / "rreo_pdf"
    pasta_saida_txt = raiz_projeto / "rreo_txt"

    # Inicia o processamento recursivo
    processar_pdfs_recursivamente(pasta_entrada_pdf, pasta_saida_txt)



Texto extraído com sucesso para: /home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/RREO_Municipal_120030_6_2023_extraido.txt
Texto extraído com sucesso para: /home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/RREO_Municipal_120030_6_2021_extraido.txt
Texto extraído com sucesso para: /home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/RREO_Estadual_12_6_2024_extraido.txt
Texto extraído com sucesso para: /home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/RREO_Municipal_120001_6_2024_extraido.txt
Texto extraído com sucesso para: /home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/RREO_Municipal_120070_6_2022_extraido.txt
Texto extraído com sucesso para: /home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/RREO_Municipal_120001_6_2021_extraido.txt
Texto extraído com sucesso para: /home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/RREO_Municipal_120001_6_2023_extraido.txt
Texto extraído com sucesso para: /home/mpac/Projetos/Curica/datasets_curic

# Criação do dataframe a partir dos arquivos TXT

## Teste arquivo individual

In [7]:
import re
import pandas as pd
import os

def carregar_texto(caminho_txt):
    """Lê o conteúdo de um arquivo TXT extraído de PDF."""
    with open(caminho_txt, "r", encoding="utf-8") as f:
        return f.read()

def extrair_dados_rreo(texto, municipio, ano):
    """
    Extrai dados principais do RREO a partir de texto bruto extraído com PyMuPDF.

    Retorna:
        dicionário com dados extraídos
    """
    dados = {
        "Município": municipio,
        "Ano": ano,
        "Receita de Impostos (Item 3)": None
    }

    # Captura específica do campo "3- TOTAL DA RECEITA RESULTANTE DE IMPOSTOS (1 + 2)"
    # Ajustado para funcionar mesmo com quebra de linha entre título e valores
    match_impostos = re.search(
        r"3- TOTAL DA RECEITA RESULTANTE DE IMPOSTOS \(1 \+ 2\)\s*(\d{1,3}(?:\.\d{3})*,\d{2})\s*(\d{1,3}(?:\.\d{3})*,\d{2})",
        texto,
        re.IGNORECASE
    )
    if match_impostos:
        dados["Receita de Impostos (Item 3)"] = match_impostos.group(2)  # Pega o segundo valor

    return dados

def processar_arquivos_pasta(pasta_txt, ano):
    """
    Itera sobre os arquivos .txt da pasta e extrai os dados de todos os relatórios.
    """
    lista_dados = []

    for nome_arquivo in os.listdir(pasta_txt):
        if nome_arquivo.endswith(".txt"):
            caminho = os.path.join(pasta_txt, nome_arquivo)
            texto = carregar_texto(caminho)

            # Extrai nome do município do nome do arquivo, ex: "rreo_acrelandia_2024_extraido.txt"
            municipio = nome_arquivo.split("_")[1].capitalize()

            dados = extrair_dados_rreo(texto, municipio, ano)
            lista_dados.append(dados)

    return pd.DataFrame(lista_dados)


# =========================
# EXEMPLO DE EXECUÇÃO
# =========================
if __name__ == "__main__":
    # Caminho da pasta contendo os arquivos TXT extraídos
    pasta_relatorios = "/home/mpac/Projetos/Curica/datasets_curica/siope/"
    ano_referencia = 2024

    df_rreo = processar_arquivos_pasta(pasta_relatorios, ano_referencia)

    # Salvar como CSV
    caminho_saida = os.path.join(pasta_relatorios, "rreo_dados_extraidos.csv")
    df_rreo.to_csv(caminho_saida, index=False, encoding="utf-8-sig", sep=';')
    print(f"Dados extraídos com sucesso. Arquivo salvo em '{caminho_saida}'.")


Dados extraídos com sucesso. Arquivo salvo em '/home/mpac/Projetos/Curica/datasets_curica/siope/rreo_dados_extraidos.csv'.


## Teste recursivo

In [1]:
import re
import pandas as pd
from pathlib import Path
import unicodedata

# Dicionário de código IBGE para nome dos municípios
CODIGO_IBGE_NOMES = {
    "1200013": "Acrelândia",
    "1200054": "Assis Brasil",
    "1200104": "Brasiléia",
    "1200138": "Bujari",
    "1200179": "Capixaba",
    "1200203": "Cruzeiro do Sul",
    "1200252": "Epitaciolândia",
    "1200302": "Feijó",
    "1200328": "Jordão",
    "1200336": "Mâncio Lima",
    "1200344": "Manoel Urbano",
    "1200351": "Marechal Thaumaturgo",
    "1200385": "Plácido de Castro",
    "1200393": "Porto Acre",
    "1200807": "Porto Walter",
    "1200333": "Rio Branco",
    "1200427": "Rodrigues Alves",
    "1200435": "Santa Rosa do Purus",
    "1200500": "Sena Madureira",
    "1200450": "Senador Guiomard",
    "1200690": "Tarauacá",
    "1200708": "Xapuri"
}

# Criar dicionário invertido com nomes formatados para comparação
def normalizar_texto(texto):
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII').lower().strip()

NOME_TO_CODIGO = {normalizar_texto(v): k for k, v in CODIGO_IBGE_NOMES.items()}

def carregar_texto(caminho_txt):
    """Lê o conteúdo de um arquivo TXT extraído de PDF."""
    with open(caminho_txt, "r", encoding="utf-8") as f:
        return f.read()

def extrair_dados_rreo(texto):
    """
    Extrai dados principais do RREO a partir de texto bruto extraído com PyMuPDF.

    Retorna:
        dicionário com dados extraídos
    """
    dados = {
        "Cod_IBGE": None,
        "Nome": None,
        "Ano": None,
        "Receita_Impostos": None,
        "20%_Fundeb": None,
        "Mínimo_MDE": None,
        "Receita_Total_Fundeb": None,
        "Receita_VAAF": None,
        "Receita_VAAT": None,
        "Receita_VAAR": None,
        "Despesa_Total_Fundeb": None,
        "Despesas_Fundeb_Prof_Educacao_Total": None,
        "Despesas_Fundeb_Prof_Infantil": None,
        "Despesas_Fundeb_Prof_Fundamental": None,
        "%_Despesas_Fundeb_Prof_Educacao": None
    }

    # Extrair nome do município (aparece após "MUNICÍPIOS\n")
    match_nome = re.search(r"MUNIC[ÍI]PIOS\s*\n([A-ZÇÀ-Ú ]+?)\s*-\s*AC", texto, re.IGNORECASE)
    if match_nome:
        nome_extraido = match_nome.group(1).strip().title()
        nome_normalizado = normalizar_texto(nome_extraido)
        dados["Nome"] = CODIGO_IBGE_NOMES.get(NOME_TO_CODIGO.get(nome_normalizado))
        dados["Cod_IBGE"] = NOME_TO_CODIGO.get(nome_normalizado)

    # Extrair ano
    match_ano = re.search(r"Per[ií]odo de Refer[eê]ncia:\s*\dº Bimestre\/(\d{4})", texto)
    if match_ano:
        dados["Ano"] = int(match_ano.group(1))

    # Extrair Receita de Impostos
    match_impostos = re.search(
        r"3- TOTAL DA RECEITA RESULTANTE DE IMPOSTOS \(1 \+ 2\)\s*(\d{1,3}(?:\.\d{3})*,\d{2})\s*(\d{1,3}(?:\.\d{3})*,\d{2})",
        texto,
        re.IGNORECASE
    )
    if match_impostos:
        dados["Receita_Impostos"] = match_impostos.group(2)

    # Extrair Repasse de 20% do Fundeb (aceita variações e dígitos soltos ao final)
    match_20_fundeb = re.search(
        r"""4-\s*TOTAL\s+DESTINADO\s+AO\s+FUNDEB\s*-\s*
            (equivalente\s+a\s+)?20%\s+DE\s+
            \(\(2\.1\.1\)\s*\+\s*\(2\.2\)\s*\+\s*\(2\.3\)\s*\+\s*\(2\.4\)\s*\+\s*\(2\.5\)
            (?:\s*\+\s*\(2\.7\))?
            \)\)*\s*\d*\s*(?:\r?\n\d+)*\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )
    if match_20_fundeb:
        dados["20%_Fundeb"] = match_20_fundeb.group(3)

    # Extrair valor mínimo a ser aplicado em MDE (com ou sem variações no cabeçalho e nos itens)
    match_mde = re.search(
        r"""5-\s*VALOR\s+M[ÍI]NIMO\s+A\s+SER\s+APLICADO\s+
            (EM\s+MDE\s+)?AL[ÉE]M\s+DO\s+VALOR\s+DESTINADO\s+AO\s+FUNDEB\s*-\s*
            5%\s+DE\s+\(\(2\.1\.1\)\s*\+\s*\(2\.2\)\s*\+\s*\(2\.3\)\s*\+\s*\(2\.4\)\s*\+\s*\(2\.5\)
            (?:\s*\+\s*\(2\.7\))?\)\s*\+\s*
            25%\s+DE\s+\(\(1\.1\)\s*\+\s*\(1\.2\)\s*\+\s*\(1\.3\)\s*\+\s*\(1\.4\)\s*\+\s*\(2\.1\.2\)\s*\+\s*\(2\.6\)
            (?:\s*\+\s*\(2\.7\))?\)\s*\)*\d*\s*(?:\r?\n\d+)*\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )
    if match_mde:
        dados["Mínimo_MDE"] = match_mde.group(3)

    # Extrair Receita Total do Fundeb (Item 9 - primeiro valor monetário após o título)
    match_fundeb_total = re.search(
        r"""9-\s*TOTAL\s+DOS\s+RECURSOS\s+DO\s+FUNDEB\s+DISPON[ÍI]VEIS\s+PARA\s+UTILIZA[ÇC][ÃA]O\s*
            \(6\s*\+\s*8\)\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})  # Primeiro valor
        """,
        texto,
        re.IGNORECASE | re.VERBOSE
    )
    if match_fundeb_total:
        dados["Receita_Total_Fundeb"] = match_fundeb_total.group(1)

    # Extrair valor do repasse do VAAF (6.2 - segundo valor monetário)
    match_vaaf = re.search(
        r"""6\.2-\s*FUNDEB\s*-\s*Complementa[çc][ãa]o\s+da\s+Uni[aã]o\s*-\s*VAAF\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE
    )
    if match_vaaf:
        dados["Receita_VAAF"] = match_vaaf.group(2)

    # Extrair valor do repasse do VAAT (6.3 - segundo valor monetário)
    match_vaat = re.search(
        r"""6\.3-\s*FUNDEB\s*-\s*Complementa[çc][ãa]o\s+da\s+Uni[aã]o\s*-\s*VAAT\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE
    )
    if match_vaat:
        dados["Receita_VAAT"] = match_vaat.group(2)

    # Extrair valor do repasse do VAAR (6.4 - segundo valor monetário)
    match_vaar = re.search(
        r"""6\.4-\s*FUNDEB\s*-\s*Complementa[çc][ãa]o\s+da\s+Uni[aã]o\s*-\s*VAAR\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE
    )
    if match_vaar:
        dados["Receita_VAAR"] = match_vaar.group(2)
    else:
        dados["Receita_VAAR"] = "0,00"

    # Extrair Despesa Total com Recursos do Fundeb (versão 1: item 12 - 4º valor)
    match_despesa_12 = re.search(
        r"""12-\s*TOTAL\s+DAS\s+DESPESAS\s*
            COM\s+RECURSOS\s+DO\s+FUNDEB\s*
            \(10\s*\+\s*11\)\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})\s+
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    # Versão 2: item 10 - com múltiplas quebras de linha e valores fragmentados
    match_despesa_10 = re.search(
        r"""10-\s*TOTAL\s+DAS\s+DESPESAS\s+COM\s+RECURSOS\s+DO\s+FUNDEB\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_despesa_12:
        dados["Despesa_Total_Fundeb"] = match_despesa_12.group(4)
    elif match_despesa_10:
        dados["Despesa_Total_Fundeb"] = match_despesa_10.group(4)

    # Extrair despesas com profissionais da educação básica (versões 10 e 10.1, quarto valor numérico)
    match_prof_educ_10 = re.search(
        r"""10-\s*PROFISSIONAIS\s+DA\s+EDUCA[ÇC][ÃA]O\s+B[ÁA]SICA\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_prof_educ_101 = re.search(
        r"""10\.1-\s*PROFISSIONAIS\s+DA\s+EDUCA[ÇC][ÃA]O\s+B[ÁA]SICA\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_prof_educ_10:
        dados["Despesas_Fundeb_Prof_Educacao_Total"] = match_prof_educ_10.group(4)
    elif match_prof_educ_101:
        dados["Despesas_Fundeb_Prof_Educacao_Total"] = match_prof_educ_101.group(4)

    # Extrair despesas com profissionais da Educação Infantil (4º valor após títulos 10.1 ou 10.1.1)
    match_inf_101 = re.search(
        r"""10\.1-\s*Educa[çc][ãa]o\s+Infantil\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_inf_1011 = re.search(
        r"""10\.1\.1\s*-\s*Educa[çc][ãa]o\s+Infantil\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_inf_101:
        dados["Despesas_Fundeb_Prof_Infantil"] = match_inf_101.group(4)
    elif match_inf_1011:
        dados["Despesas_Fundeb_Prof_Infantil"] = match_inf_1011.group(4)

    # Extrair despesas com profissionais do Ensino Fundamental (4º valor após os títulos 10.2 ou 10.1.2)
    match_fund_102 = re.search(
        r"""10\.2-\s*Ensino\s+Fundamental\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_fund_1012 = re.search(
        r"""10\.1\.2-\s*Ensino\s+Fundamental\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_fund_102:
        dados["Despesas_Fundeb_Prof_Fundamental"] = match_fund_102.group(4)
    elif match_fund_1012:
        dados["Despesas_Fundeb_Prof_Fundamental"] = match_fund_1012.group(4)

    # Extrair percentual das despesas do Fundeb com profissionais da educação (4º valor após item 19 ou 15)
    match_pct_19 = re.search(
        r"""19-\s*M[ÍI]NIMO\s+DE\s+70%\s+DO\s+FUNDEB\s+NA\s+REMUNERA[ÇC][ÃA]O\s+
            DOS\s+PROFISSIONAIS\s+DA\s+EDUCA[ÇC][ÃA]O\s+B[ÁA]SICA\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    match_pct_15 = re.search(
        r"""15-\s*M[ÍI]NIMO\s+DE\s+70%\s+DO\s+FUNDEB\s+NA\s+REMUNERA[ÇC][ÃA]O\s+
            DOS\s+PROFISSIONAIS\s+DA\s+EDUCA[ÇC][ÃA]O\s+B[ÁA]SICA\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})\s*
            (\d{1,3}(?:\.\d{3})*,\d{2})
        """,
        texto,
        re.IGNORECASE | re.VERBOSE | re.DOTALL
    )

    if match_pct_19:
        dados["%_Despesas_Fundeb_Prof_Educacao"] = match_pct_19.group(4)
    elif match_pct_15:
        dados["%_Despesas_Fundeb_Prof_Educacao"] = match_pct_15.group(4)

  
  
    # Não excluir essa linha da função!
    return dados


def processar_arquivos_pasta(pasta_txt):
    """
    Itera sobre os arquivos .txt da pasta e extrai os dados de todos os relatórios.
    Retorna um DataFrame contendo os dados válidos extraídos.
    """
    lista_dados = []
    pasta_txt = Path(pasta_txt)

    for caminho in pasta_txt.glob("*.txt"):
        texto = carregar_texto(caminho)
        try:
            dados = extrair_dados_rreo(texto)
            if dados and dados.get("Cod_IBGE") and dados.get("Nome"):
                lista_dados.append(dados)
            else:
                print(f"Aviso: Nome ou código IBGE não encontrado ou erro de parsing em {caminho.name}")
        except Exception as e:
            print(f"Erro ao processar o arquivo {caminho.name}: {e}")
            continue

    df = pd.DataFrame(lista_dados)

    # Ordena apenas se não estiver vazio
    if not df.empty:
        df = df.sort_values(by=["Nome", "Ano"], ascending=[True, True])

    return df


# =========================
# EXEMPLO DE EXECUÇÃO
# =========================
if __name__ == "__main__":
    pasta_arquivos_txt = Path("/home/mpac/Projetos/Curica/datasets_curica/siope/rreo_txt/")
    pasta_dataframe = Path("/home/mpac/Projetos/Curica/datasets_curica/siope/rreo_df/")

    df_rreo = processar_arquivos_pasta(pasta_arquivos_txt)

    # Ordena as colunas do dataframe
    df_rreo = df_rreo.sort_values(by=["Nome", "Ano"], ascending=[True, True])

    # Exporta para CSV
    caminho_saida = pasta_dataframe / "df_rreo_teste.csv"
    df_rreo.to_csv(caminho_saida, index=False, encoding="utf-8-sig", sep=';')
    print(f"Dados extraídos com sucesso. Arquivo salvo em '{caminho_saida}'.")


Dados extraídos com sucesso. Arquivo salvo em '/home/mpac/Projetos/Curica/datasets_curica/siope/rreo_df/df_rreo_teste.csv'.
